In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

base_path = '/content/drive/MyDrive/Banco com zip/'

for year in range(2000, 2025):
    zip_file_path = os.path.join(base_path, str(year) + '.zip')
    if os.path.exists(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            extraction_path = os.path.join('diretorio_destino', str(year))
            zip_ref.extractall(extraction_path)
            print(f"Files from {zip_file_path} extracted to {extraction_path}")
    else:
        print(f"Zip file not found for year {year}: {zip_file_path}")

In [ ]:
import os
import polars as pl
import sqlite3

def map_polars_to_sql_type(polars_type):
    if polars_type == pl.Int32 or polars_type == pl.Int64:
        return "INTEGER"
    elif polars_type == pl.Float32 or polars_type == pl.Float64:
        return "REAL"
    else:
        return "TEXT"

def processar_e_salvar_no_sql(ano, chunk_size=1000):
    # conectando ao banco de dados SQLite (será criado se não existir)
    db_path = '/content/dados_anos.db'
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    diretorio_destino = f'/content/diretorio_destino/{ano}/{ano}' if ano < 2020 else f'/content/diretorio_destino/{ano}'

    arquivos_csv = [f for f in os.listdir(diretorio_destino) if f.lower().endswith(".csv")]

    tabela_criada = False  # Flag para garantir q a tabela seja criada só uma vez

    for arquivo in arquivos_csv:
        caminho_completo = os.path.join(diretorio_destino, arquivo)
        estacao = arquivo.split('_')[3]

        # captura as colunas e tipos
        first_chunk = pl.read_csv(
            caminho_completo,
            skip_rows=8,
            separator=";",
            decimal_comma=True,
            encoding="ISO-8859-1",
            infer_schema_length=10000
        ).with_columns(pl.lit(estacao).alias("estacao"))

        colunas = first_chunk.columns
        tipos = [map_polars_to_sql_type(dtype) for dtype in first_chunk.dtypes]

        if not tabela_criada:
            cursor.execute(f"DROP TABLE IF EXISTS dados_{ano}")

            colunas_sql = ", ".join([f'"{col}" {tipo}' for col, tipo in zip(colunas, tipos)])
            cursor.execute(f"CREATE TABLE dados_{ano} ({colunas_sql})")
            conn.commit()
            tabela_criada = True
        for chunk_df in pl.read_csv(
                caminho_completo,
                skip_rows=8,
                separator=";",
                decimal_comma=True,
                encoding="ISO-8859-1",
                infer_schema_length=10000,
                ignore_errors=True,
                null_values="-9999",
        ).iter_slices(chunk_size):
            chunk_df = chunk_df.with_columns(pl.lit(estacao).alias("estacao"))

            registros = chunk_df.to_numpy().tolist()
            placeholders = ', '.join(['?' for _ in colunas])

            cursor.executemany(
                f'INSERT INTO dados_{ano} VALUES ({placeholders})',
                registros
            )

    conn.commit()
    conn.close()
    print(f"Dados do ano {ano} processados e salvos no banco de dados com sucesso!")

for ano in range(2000, 2025):
    processar_e_salvar_no_sql(ano, chunk_size=1000)
    print(f"Data para o ano {ano} processada e armazenada com sucesso!")


In [ ]:
import pandas as pd

In [ ]:
db_path = '/content/dados_anos.db'
conn = sqlite3.connect(db_path)

In [ ]:
pd.read_sql_query("""SELECT * FROM dados_2000""", conn)

In [ ]:
pd.read_sql_query("""SELECT "DATA (YYYY-MM-DD)", `HORA (UTC)`,`TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)`, estacao FROM dados_2000
ORDER BY `TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)` DESC LIMIT 1""", conn)